In [ ]:
pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 2.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install flask_ngrok

In [ ]:
import json
import os
import re
from collections import defaultdict

def clean_text(text):
    # Remove extra whitespace characters
    text = re.sub(r'\s+', ' ', text).strip()
    # Normalize punctuation (this is just a simple example, you may need more complex rules)
    text = text.replace('，', ',').replace('。', '.')
    return text

def are_similar(text1, text2, threshold=0.8):
    # This is a simple similarity check, you may need a more complex algorithm
    words1 = set(text1.lower().split())
    words2 = set(text2.lower().split())
    intersection = words1.intersection(words2)
    return len(intersection) / max(len(words1), len(words2)) > threshold

# Define input file paths
input_files = ['/content/drive/MyDrive/chatbot/merged_qa_dataset.json']

# Define output file path
output_file = '/content/cleaned_data'

# Initialize merged data list and question dictionary
merged_data = []
question_dict = defaultdict(list)

# Iterate over each input file
for file_path in input_files:
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
        if isinstance(data, list):
            for item in data:
                question = clean_text(item.get('question', ''))
                answer = clean_text(item.get('answer', ''))
                category = item.get('category', '').strip()

                if question and answer:
                    question_dict[question].append((answer, category))
        elif isinstance(data, dict):
            question = clean_text(data.get('question', ''))
            answer = clean_text(data.get('answer', ''))
            category = data.get('category', '').strip()

            if question and answer:
                question_dict[question].append((answer, category))

# Merge similar questions
final_data = []
for question, answers in question_dict.items():
    similar_found = False
    for existing in final_data:
        if are_similar(question, existing['question']):
            # If a similar question is found, merge answers and categories
            existing['answers'].extend(answers)
            similar_found = True
            break
    if not similar_found:
        final_data.append({
            'question': question,
            'answers': answers
        })

# Reassign IDs and format output
for index, item in enumerate(final_data, start=1):
    merged_data.append({
        'id': index,
        'question': item['question'],
        'answer': '; '.join(set(answer for answer, _ in item['answers'])),
        'category': '; '.join(set(category for _, category in item['answers'] if category))
    })

# Write the merged data to a new file
with open(output_file, 'w', encoding='utf-8') as outfile:
    json.dump(merged_data, outfile, ensure_ascii=False, indent=2)

print(f"Merging complete, processed {len(merged_data)} unique entries. Output file: {output_file}")

Merging complete, processed 166 unique entries. Output file: /content/cleaned_data


In [ ]:
import json
import os
import re
from collections import defaultdict
import torch
from sentence_transformers import SentenceTransformer, util
from sklearn.cluster import KMeans
import pandas as pd

def clean_text(text):
    # Remove extra whitespace characters
    text = re.sub(r'\s+', ' ', text).strip()
    # Normalize punctuation (this is just a simple example, you may need more complex rules)
    text = text.replace('，', ',').replace('。', '.')
    return text

def are_similar(text1, text2, threshold=0.8):
    # This is a simple similarity check, you may need a more complex algorithm
    words1 = set(text1.lower().split())
    words2 = set(text2.lower().split())
    intersection = words1.intersection(words2)
    return len(intersection) / max(len(words1), len(words2)) > threshold

# Define input file paths
input_files = ['/content/drive/MyDrive/chatbot/merged_qa_dataset.json']

# Define output file path
output_file = '/content/cleaned_data'

# Initialize merged data list and question dictionary
merged_data = []
question_dict = defaultdict(list)

# Iterate over each input file
for file_path in input_files:
    with open(file_path, 'r', encoding='utf-8') as file:
        data = json.load(file)
        if isinstance(data, list):
            for item in data:
                question = clean_text(item.get('question', ''))
                answer = clean_text(item.get('answer', ''))
                category = item.get('category', '').strip()

                if question and answer:
                    question_dict[question].append((answer, category))
        elif isinstance(data, dict):
            question = clean_text(data.get('question', ''))
            answer = clean_text(data.get('answer', ''))
            category = data.get('category', '').strip()

            if question and answer:
                question_dict[question].append((answer, category))

# Merge similar questions
final_data = []
for question, answers in question_dict.items():
    similar_found = False
    for existing in final_data:
        if are_similar(question, existing['question']):
            # If a similar question is found, merge answers and categories
            existing['answers'].extend(answers)
            similar_found = True
            break
    if not similar_found:
        final_data.append({
            'question': question,
            'answers': answers
        })

# Reassign IDs and format output
for index, item in enumerate(final_data, start=1):
    merged_data.append({
        'id': index,
        'question': item['question'],
        'answer': '; '.join(set(answer for answer, _ in item['answers'])),
        'category': '; '.join(set(category for _, category in item['answers'] if category))
    })

# Write the merged data to a new file
with open(output_file, 'w', encoding='utf-8') as outfile:
    json.dump(merged_data, outfile, ensure_ascii=False, indent=2)

print(f"Merging complete, processed {len(merged_data)} unique entries. Output file: {output_file}")

# Load data
with open('/content/cleaned_data', 'r', encoding='utf-8') as file:
    qa_data = json.load(file)

# Load customer information
customer_info = pd.read_csv('/content/drive/MyDrive/chatbot/dummy_data (1).csv')
customer_info.set_index('Account Number', inplace=True)

# Extract questions and answers
questions = [item['question'] for item in qa_data]
answers = [item['answer'] for item in qa_data]

# Load pre-trained BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Encode all questions
question_embeddings = model.encode(questions, convert_to_tensor=True)

# Use K-means for simple intent clustering
num_clusters = 5
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(question_embeddings.cpu().numpy())

class QAAgent:
    def get_intent(self, user_question):
        user_embedding = model.encode(user_question, convert_to_tensor=True)
        intent = kmeans.predict(user_embedding.cpu().numpy().reshape(1, -1))[0]
        return f"Intent_{intent}"

    def get_answer(self, user_question, conversation_history):
        user_embedding = model.encode(user_question, convert_to_tensor=True)
        cos_scores = util.cos_sim(user_embedding, question_embeddings)[0]
        top_result = torch.topk(cos_scores, k=1)
        most_similar_index = top_result.indices[0].item()
        answer = answers[most_similar_index]

        if conversation_history:
            answer = f"Based on our previous conversation about {', '.join(conversation_history)}, {answer}"

        return answer

    def get_personal_answer(self, user_question, customer_id):
        print(f"Debug: Accessing personal info for account {customer_id}")
        print(f"Debug: customer_id type: {type(customer_id)}")

        # 尝试将 customer_id 转换为整数
        try:
            customer_id = int(customer_id)
        except ValueError:
            print(f"Debug: Failed to convert customer_id to int: {customer_id}")
            return "I'm sorry, there was an error processing your account number."

        if customer_id in customer_info.index:
            info = customer_info.loc[customer_id]
            print(f"Debug: Found customer info: {info}")
            if "balance" in user_question.lower():
                return f"Your current account balance is ${info['Current Balance']}. Your sub-account balance is ${info['Sub_acc_Current_Balance']}."
            elif "loan amount" in user_question.lower():
                return f"Your original loan amount was ${info['Original Loan Amount']}."
            elif "maturity date" in user_question.lower():
                return f"Your loan maturity date is {info['Maturity Date']}."
            elif "name" in user_question.lower():
                return f"Your name is {info['First Name']} {info['Last Name']}."
            elif "payment frequency" in user_question.lower():
                return f"Your main account payment frequency is {info['Payment Frequency']}. Your sub-account payment frequency is {info['Sub_acc_Payment_Frequency']}."
            elif "prepayment" in user_question.lower():
                return f"Your prepayment amount is ${info['Prepayment']}."
            elif "interest type" in user_question.lower():
                return f"Your sub-account interest type is {info['Sub_acc_Interest_Type']}."
            else:
                return "I'm sorry, I can't answer this specific personal question. Please ask about your balance, loan amount, maturity date, name, payment frequency, prepayment, or interest type."
        else:
            print(f"Debug: Customer ID {customer_id} not found in customer_info")
            print(f"Debug: customer_info index: {customer_info.index}")
            return "I'm sorry, I couldn't find your personal information."

class VerificationAgent:
    def verify_answer(self, question, answer):
        keywords = question.lower().split()
        if any(keyword in answer.lower() for keyword in keywords):
            return True, "The answer appears to be relevant to the question."
        else:
            return False, "The answer may not be directly related to the question. Please verify."

    def verify_identity(self, account_number, first_name, last_name):
        try:
            account_number = int(account_number)  # 尝试将账号转换为整数
        except ValueError:
            return False, "Invalid account number format."

        if account_number in customer_info.index:
            actual_info = customer_info.loc[account_number]
            print(f"Debug: Actual info - {actual_info['First Name']}, {actual_info['Last Name']}")
            print(f"Debug: Input - {first_name}, {last_name}")
            if (first_name.lower().strip() == str(actual_info['First Name']).lower().strip() and
                last_name.lower().strip() == str(actual_info['Last Name']).lower().strip()):
                return True, "Identity verification successful."
            else:
                return False, "Identity verification failed. Please try again."
        else:
            return False, "Account number not found."

class ChatbotManager:
    def __init__(self):
        self.qa_agent = QAAgent()
        self.verification_agent = VerificationAgent()
        self.conversation_history = []
        self.current_account_number = None
        self.current_first_name = None
        self.is_verified = False
        self.verification_stage = 0

    def process_input(self, user_input):
        intent = self.qa_agent.get_intent(user_input)
        print(f"Detected intent: {intent}")

        personal_keywords = ["balance", "loan", "maturity", "name", "payment", "prepayment", "interest"]
        is_personal_question = any(keyword in user_input.lower() for keyword in personal_keywords)

        if is_personal_question or self.verification_stage > 0:
            if not self.is_verified:
                if self.verification_stage == 0:
                    self.verification_stage = 1
                    return "This is a personal question. Please verify your identity first. Enter your account number:"
                elif self.verification_stage == 1:
                    self.current_account_number = user_input
                    print(f"Debug: Set current_account_number to {self.current_account_number}")
                    self.verification_stage = 2
                    return "Please enter your first name for identity verification."
                elif self.verification_stage == 2:
                    self.current_first_name = user_input
                    self.verification_stage = 3
                    return "Please enter your last name for identity verification."
                elif self.verification_stage == 3:
                    is_verified, message = self.verification_agent.verify_identity(
                        self.current_account_number, self.current_first_name, user_input)
                    if is_verified:
                        self.is_verified = True
                        self.verification_stage = 0
                        return "Identity verification successful. You can now ask personal questions."
                    else:
                        self.current_account_number = None
                        self.current_first_name = None
                        self.verification_stage = 0
                        return "Identity verification failed. Please start the verification process again."

        if self.is_verified and is_personal_question:
            print(f"Debug: Verified user asking personal question. Account number: {self.current_account_number}")  # 添加调试信息
            answer = self.qa_agent.get_personal_answer(user_input, self.current_account_number)
        else:
            answer = self.qa_agent.get_answer(user_input, self.conversation_history)

        is_verified, verification_message = self.verification_agent.verify_answer(user_input, answer)

        if is_verified:
            response = f"Bot: {answer}\n(Verification: {verification_message})"
        else:
            response = f"Bot: I'm not entirely sure, but here's what I found: {answer}\n(Verification: {verification_message})"

        self.conversation_history.append(intent)
        if len(self.conversation_history) > 3:
            self.conversation_history.pop(0)

        return response

# Chatbot main loop
print("Welcome to the multi-agent BERT-powered bank chatbot! Type 'exit' to end the conversation.")
chatbot_manager = ChatbotManager()

while True:
    user_input = input("Your question: ")
    if user_input.lower() == 'exit':
        print("Thank you for using the chatbot. Goodbye!")
        break

    response = chatbot_manager.process_input(user_input)
    print(response)

# 在加载 customer_info 后立即添加这些行
print("Debug: customer_info index type:", type(customer_info.index[0]))
print("Debug: customer_info index:", customer_info.index)
print("Debug: customer_info columns:", customer_info.columns)
print("Debug: First few rows of customer_info:")
print(customer_info.head())


Merging complete, processed 166 unique entries. Output file: /content/cleaned_data


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Welcome to the multi-agent BERT-powered bank chatbot! Type 'exit' to end the conversation.
Detected intent: Intent_3
This is a personal question. Please verify your identity first. Enter your account number:
Detected intent: Intent_3
Debug: Set current_account_number to 3753712
Please enter your first name for identity verification.
Detected intent: Intent_3
Please enter your last name for identity verification.
Detected intent: Intent_3
Debug: Actual info - Gregg, Holmes
Debug: Input - Gregg, Holmes
Identity verification successful. You can now ask personal questions.
Detected intent: Intent_3
Debug: Verified user asking personal question. Account number: 3753712
Debug: Accessing personal info for account 3753712
Debug: customer_id type: <class 'str'>
Debug: Found customer info: First Name                        Gregg
Last Name                        Holmes
Maturity Date                23/08/2025
Original Loan Amount          404474.51
Current Balance               234756.92
Prepaymen